In [ ]:
Biology2e-WEB_Excerpt.pdf

In [5]:
import PyPDF2
import json

def main(file_path, output_path):
    with open(file_path, 'rb') as f:
        reader = PyPDF2.PdfReader(f)
        num_pages = len(reader.pages)
        all_text = ''
        for page_num in range(num_pages):
            page = reader.pages[page_num]
            all_text += page.extract_text()

    # Process `all_text` to generate your JSON data
    # For example, convert it to a dictionary or list
    
    # Example: Converting to a dictionary
    json_data = {
        "text": all_text,
        "pages": num_pages
    }

    # Writing JSON data to file
    with open(output_path, 'w', encoding='utf-8') as json_file:
        json.dump(json_data, json_file, ensure_ascii=False)

    print(f"Data successfully written to {output_path}")

if __name__ == "__main__":
    input_pdf = "Biology2e-WEB_Excerpt.pdf"  # Replace with your PDF file path
    output_json = "output1.json"  # Replace with your desired output file path
    main(input_pdf, output_json)


Data successfully written to output1.json


In [8]:
from pydantic import BaseModel, FilePath

class PdfToJsonInput(BaseModel):
    file_path: FilePath
    output_path: FilePath

import PyPDF2
import json

def pdf_to_json(input_data: PdfToJsonInput):
    file_path = input_data.file_path
    output_path = input_data.output_path

    with open(file_path, 'rb') as f:
        reader = PyPDF2.PdfReader(f)
        num_pages = len(reader.pages)
        all_text = ''
        for page_num in range(num_pages):
            page = reader.pages[page_num]
            all_text += page.extract_text()

    # Process `all_text` to generate your JSON data
    json_data = {
        "text": all_text,
        "pages": num_pages
    }

    # Writing JSON data to file
    with open(output_path, 'w', encoding='utf-8') as json_file:
        json.dump(json_data, json_file, ensure_ascii=False)

    print(f"Data successfully written to {output_path}")

if __name__ == "__main__":
    # Example usage:
    input_data = PdfToJsonInput(file_path="Biology2e-WEB_Excerpt.pdf", output_path="output1.json")
    pdf_to_json(input_data)


Data successfully written to output1.json


In [9]:
from pydantic import BaseModel, FilePath

class PdfToJsonInput(BaseModel):
    file_path: FilePath
    output_path: FilePath

import PyPDF2
import json

def pdf_to_json(input_data: PdfToJsonInput):
    file_path = input_data.file_path
    output_path = input_data.output_path

    with open(file_path, 'rb') as f:
        reader = PyPDF2.PdfReader(f)
        num_pages = len(reader.pages)
        all_text = ''
        for page_num in range(num_pages):
            page = reader.pages[page_num]
            all_text += page.extract_text()

    # Process `all_text` to generate your JSON data
    json_data = {
        "text": all_text,
        "pages": num_pages
    }

    # Writing JSON data to file
    with open(output_path, 'w', encoding='utf-8') as json_file:
        json.dump(json_data, json_file, ensure_ascii=False)

    print(f"Data successfully written to {output_path}")

if __name__ == "__main__":
    # Example usage:
    input_data = PdfToJsonInput(file_path="Biology2e-WEB_Excerpt.pdf", output_path="output1.json")
    pdf_to_json(input_data)


Data successfully written to output1.json


In [10]:
import fitz  # PyMuPDF
import json

def pdf_to_json(input_data: PdfToJsonInput):
    file_path = input_data.file_path
    output_path = input_data.output_path

    doc = fitz.open(file_path)
    num_pages = doc.page_count
    all_data = []

    for page_num in range(num_pages):
        page_data = {
            "page_number": page_num + 1,
            "text": '',
            "images": []
        }
        page = doc.load_page(page_num)
        
        # Extract text
        page_data["text"] = page.get_text()
        
        # Extract images
        image_list = page.get_images(full=True)
        for img_index, img in enumerate(image_list):
            image_bytes = doc.extract_image(img[0])
            image_data = {
                "image_index": img_index + 1,
                "image_data": image_bytes
            }
            page_data["images"].append(image_data)
        
        all_data.append(page_data)

    doc.close()

    # Writing JSON data to file
    with open(output_path, 'w', encoding='utf-8') as json_file:
        json.dump(all_data, json_file, ensure_ascii=False)

    print(f"Data successfully written to {output_path}")

if __name__ == "__main__":
    # Example usage:
    input_data = PdfToJsonInput(file_path="Biology2e-WEB_Excerpt.pdf", output_path="output2.json")
    pdf_to_json(input_data)


ValidationError: 1 validation error for PdfToJsonInput
output_path
  Path does not point to a file [type=path_not_file, input_value='output2.json', input_type=str]

In [13]:
from pydantic import BaseModel, FilePath, validator
from typing import List, Dict
import os

class PdfToJsonInput(BaseModel):
    file_path: FilePath
    output_path: FilePath

    @validator('output_path')
    def validate_output_path(cls, v):
        if not os.path.isfile(v):
            try:
                with open(v, 'w'):  # Create the file if it doesn't exist
                    pass
            except IOError:
                raise ValueError(f"Unable to create file at {v}")
        return v

import fitz  # PyMuPDF
import json

def pdf_to_json(input_data: PdfToJsonInput):
    file_path = input_data.file_path
    output_path = input_data.output_path

    doc = fitz.open(file_path)
    num_pages = doc.page_count
    all_data = []

    for page_num in range(num_pages):
        page_data = {
            "page_number": page_num + 1,
            "text": '',
            "images": []
        }
        page = doc.load_page(page_num)
        
        # Extract text
        page_data["text"] = page.get_text()
        
        # Extract images
        image_list = page.get_images(full=True)
        for img_index, img in enumerate(image_list):
            image_bytes = doc.extract_image(img[0])
            image_data = {
                "image_index": img_index + 1,
                "image_data": image_bytes
            }
            page_data["images"].append(image_data)
        
        all_data.append(page_data)

    doc.close()

    # Writing JSON data to file
    with open(output_path, 'w', encoding='utf-8') as json_file:
        json.dump(all_data, json_file, ensure_ascii=False)

    print(f"Data successfully written to {output_path}")

if __name__ == "__main__":
    # Example usage:
    input_data = PdfToJsonInput(file_path="Biology2e-WEB_Excerpt.pdf", output_path="output2.json")
    pdf_to_json(input_data)


c:\users\dimple\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:9: PydanticDeprecatedSince20: Pydantic V1 style `@validator` validators are deprecated. You should migrate to Pydantic V2 style `@field_validator` validators, see the migration guide for more details. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.5/migration/
  if __name__ == "__main__":


ValidationError: 1 validation error for PdfToJsonInput
output_path
  Path does not point to a file [type=path_not_file, input_value='output2.json', input_type=str]

In [14]:
from pydantic import BaseModel, FilePath, Field
import os

class PdfToJsonInput(BaseModel):
    file_path: FilePath
    output_path: FilePath

    @staticmethod
    def validate_output_path(v):
        if not os.path.isfile(v):
            try:
                with open(v, 'w'):  # Create the file if it doesn't exist
                    pass
            except IOError:
                raise ValueError(f"Unable to create file at {v}")
        return v

import fitz  # PyMuPDF
import json

def pdf_to_json(input_data: PdfToJsonInput):
    file_path = input_data.file_path
    output_path = input_data.output_path

    doc = fitz.open(file_path)
    num_pages = doc.page_count
    all_data = []

    for page_num in range(num_pages):
        page_data = {
            "page_number": page_num + 1,
            "text": '',
            "images": []
        }
        page = doc.load_page(page_num)
        
        # Extract text
        page_data["text"] = page.get_text()
        
        # Extract images
        image_list = page.get_images(full=True)
        for img_index, img in enumerate(image_list):
            image_bytes = doc.extract_image(img[0])
            image_data = {
                "image_index": img_index + 1,
                "image_data": image_bytes
            }
            page_data["images"].append(image_data)
        
        all_data.append(page_data)

    doc.close()

    # Writing JSON data to file
    with open(output_path, 'w', encoding='utf-8') as json_file:
        json.dump(all_data, json_file, ensure_ascii=False)

    print(f"Data successfully written to {output_path}")

if __name__ == "__main__":
    # Example usage:
    input_data = PdfToJsonInput(file_path="Biology2e-WEB_Excerpt.pdf", output_path="output2.json")
    pdf_to_json(input_data)


ValidationError: 1 validation error for PdfToJsonInput
output_path
  Path does not point to a file [type=path_not_file, input_value='output2.json', input_type=str]

In [16]:
from pydantic import BaseModel, FilePath, Field
import os

class PdfToJsonInput(BaseModel):
    file_path: FilePath
    output_path: FilePath = Field(..., description="Path to the output JSON file.")

    @validator('output_path')
    def validate_output_path(cls, v):
        if not os.path.isfile(v):
            try:
                with open(v, 'w'):  # Create the file if it doesn't exist
                    pass
            except IOError:
                raise ValueError(f"Unable to create file at {v}")
        return v

import fitz  # PyMuPDF
import json

def pdf_to_json(input_data: PdfToJsonInput):
    file_path = input_data.file_path
    output_path = input_data.output_path

    doc = fitz.open(file_path)
    num_pages = doc.page_count
    all_data = []

    for page_num in range(num_pages):
        page_data = {
            "page_number": page_num + 1,
            "text": '',
            "images": []
        }
        page = doc.load_page(page_num)
        
        # Extract text
        page_data["text"] = page.get_text()
        
        # Extract images
        image_list = page.get_images(full=True)
        for img_index, img in enumerate(image_list):
            image_bytes = doc.extract_image(img[0])
            image_data = {
                "image_index": img_index + 1,
                "image_data": image_bytes
            }
            page_data["images"].append(image_data)
        
        all_data.append(page_data)

    doc.close()

    # Writing JSON data to file
    with open(output_path, 'w', encoding='utf-8') as json_file:
        json.dump(all_data, json_file, ensure_ascii=False)

    print(f"Data successfully written to {output_path}")

if __name__ == "__main__":
    # Example usage:
    input_data = PdfToJsonInput(file_path="Biology2e-WEB_Excerpt.pdf", output_path="output.json")
    pdf_to_json(input_data)


c:\users\dimple\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:8: PydanticDeprecatedSince20: Pydantic V1 style `@validator` validators are deprecated. You should migrate to Pydantic V2 style `@field_validator` validators, see the migration guide for more details. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.5/migration/
  


TypeError: Object of type bytes is not JSON serializable

In [1]:
pip install Pillow


Note: you may need to restart the kernel to use updated packages.


In [5]:
import fitz  # PyMuPDF
import json
from pydantic import BaseModel
from typing import List, Optional
from PIL import Image
from io import BytesIO
import base64

class ImageModel(BaseModel):
    page_number: int
    image_index: int
    image_base64: str

class PageModel(BaseModel):
    page_number: int
    text: str
    images: Optional[List[ImageModel]]

class PDFModel(BaseModel):
    file_name: str
    pages: List[PageModel]

def extract_text_and_images(pdf_path: str) -> PDFModel:
    doc = fitz.open(pdf_path)
    pdf_data = PDFModel(file_name=pdf_path, pages=[])

    for page_number in range(len(doc)):
        page = doc[page_number]
        text = page.get_text("text")
        images = []

        for img_index, img in enumerate(page.get_images(full=True)):
            xref = img[0]
            base_image = doc.extract_image(xref)
            image_bytes = base_image["image"]

            # Convert image to base64
            img_base64 = base64.b64encode(image_bytes).decode('utf-8')
            images.append(ImageModel(page_number=page_number, image_index=img_index, image_base64=img_base64))

        page_data = PageModel(page_number=page_number, text=text, images=images)
        pdf_data.pages.append(page_data)

    return pdf_data

# Specify the PDF file path
pdf_file_path = 'Biology2e-WEB_Excerpt.pdf'

# Extract data and convert to JSON
pdf_data = extract_text_and_images(pdf_file_path)

# Convert to JSON using the json module
pdf_dict = pdf_data.dict()
pdf_json = json.dumps(pdf_dict, indent=4)

# Adding line spaces between different sections in the JSON string
pdf_json_with_spaces = pdf_json.replace('},', '},\n')

# Save JSON to a file
with open('output.json', 'w') as f:
    f.write(pdf_json_with_spaces)

print("PDF data has been successfully converted to JSON with line spaces and saved as 'output.json'.")

PDF data has been successfully converted to JSON with line spaces and saved as 'output.json'.


c:\users\dimple\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:53: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.5/migration/
